# Understanding your Network and its Configuration 
From *How Linux Works* by Brian Ward, Ch 9. Understanding your Network and its Configuration.

## Packets
* Computers communicate with one another by transmitting small chuncks of data across the network. Each contains a *header*, and a *payload*.
    * __Header__ - Contains the to and from Host destinations, as well as basic protocol.
    * __Payload__ - Contains application data the host wants to send (for example, HTML or image data).
    
## Network Layers
* __Application layer__ (e.g. HTTP, FTP) - Contains the “language” that applications and servers use to communicate, such as Hypertext Transfer Protocol, Secure Socket Layer (SSL), and File Transfer Protocol (FTP). Application layer protocols can often be combined. For example, SSL is commonly used in conjunction with HTTP.
* __Transport layer (protocol layer)__ (e.g. TCP, UDP) - This layer includes data integrity checking, source and destination ports, and specifications for breaking application data into packets (if the application layer has not already done so).
* __Network Layer (Internet layer)__ (Internet Protocol, IP) - Defines how to move packets from a source host to a destination host. IP is the packet transit rule set for the Internet.
* __Physical Layer (link layer or host-to-network layer)__ - Defines how to send raw data across a physical medium, such as Ethernet or a modem.

## The Internet Layer
 One of the most important aspects of the Internet layer is that it’s meant to be a software network that places no particular requirements on hardware or operating systems. The idea is that you can send and receive Internet packets over any kind of hardware, using any operating system. 
 
Internet’s topology is decentralized; it’s made up of smaller networks called __subnets__ (e.g. your LAN). The idea is that all subnets are interconnected in some way.
 
__A host can be attached to more than one subnet (a.k.a. router, or gateway). A router typically has at least two addresses, the local subnet (e.g. 10.23.2.1) and the link to the Internet. If a host is connected to multiple subnets, it has at least one IP address per subnet.__

Each Internet host has at least one numeric IP address in the form of a.b.c.d (dotted-quad sequence), such as 10.23.2.37. Technically, an IP address consists of 4 bytes (or 32 bits), abcd. Bytes a and d are numbers from 1 to 254, and b and c are numbers from 0 to 255. A computer processes IP addresses as raw bytes, even though we read and write them as a dotted-quad addresses.

### Viewing Your Computer’s IP Addresses
One host can have many IP addresses. To see the addresses that are active on your Linux machine, run 

`$ ifconfig`

There will probably be a lot of output, but it should include something like this:

```bash
eth0    Link encap:Ethernet  HWaddr 10:78:d2:eb:76:97      
        inet addr:10.23.2.4  Bcast:10.23.2.255  Mask:255.255.255.0           
        inet6 addr: fe80::1278:d2ff:feeb:7697/64 Scope:Link           
        UP BROADCAST RUNNING MULTICAST  MTU:1500  Metric:1           
        RX packets:85076006 errors:0 dropped:0 overruns:0 frame:0           
        TX packets:68347795 errors:0 dropped:0 overruns:0 carrier:0           
        collisions:0 txqueuelen:1000           
        RX bytes:86427623613 (86.4 GB)  TX bytes:23437688605 (23.4 GB)           
        Interrupt:20 Memory:fe500000-fe520000
```
The second line reports that the host is configured to have an IPv4 address (inet addr) of 10.23.2.4. On the same line, a Mask is reported as being 255.255.255.0. This is a subnet mask, which defines the subnet that an IP address belongs to.

### Subnets
You define a subnet with two pieces: a __network prefix__ and a __subnet mask__.
* __Network Prefix__ - The part that is common to all ip addresses in the subnet.
    * Example 10.23.2.0 would be the network prefix for a subnet containing the IP addresses between 
10.23.2.1 and 10.23.2.254.
* __Subnet Mask__ - Marks the bit locations in an IP address that are common to the subnet.
    * Following up with the previous example, the subnet mask would be 255.255.255.0.
    
Valid IP addresses in the example subnet would all contain the same Network Prefix and the last numbers in sequence could be anything from 1 to 254 (because of the subnet mask it has).

### Common Subnet Masks and CIDR Notation
__Classless Inter-Domain Routing (CIDR) notation__ - An alternative form of subnet representation.
* A subnet such as 10.23.2.0/255.255.255.0 is written as 10.23.2.0/24. 

To understand what this means, look at the mask in binary form. You’ll find that nearly all subnet masks are just a bunch of 1s followed by a bunch of 0s. For example, 255.255.255.0 in binary form is 24 1-bits followed by 8 0-bits. The CIDR notation identifies the subnet mask by the number of leading 1s in the subnet mask. Therefore, a combination such as 10.23.2.0/24 includes both the subnet prefix and its subnet mask. 

Dotted-Quad ----> Binary ----------------------------------------> CIDR

255.255.255.0 --> 11111111 11111111 11111111 0000000 --> 24

#### Subnet Masks

![CIDR Subnet Masks](common_cidr.png)

## Routes and the Kernel Routing Table
Connecting Internet subnets is mostly a process of identifying the hosts connected to more than one subnet. A host connected to a local network can directly reach any hosts on that network. To reach hosts on the rest of the Internet, it must communicate through its router. 

__How does the Linux kernel distinguish between these two different kinds of destinations?__

__It uses a destination configuration called a routing table to determine its routing behavior.__ To show the routing table, use the route -n command. Here’s what you might see for a simple host such as 10.23.2.4: 

`$ route -n`

\* *The -n option tells route to show IP addresses instead of showing hosts and networks by name. This is an important option to remember because you’ll be able to use it in other network-related commands such as netstat.*

```bash
Kernel IP routing table 
Destination     Gateway     Genmask       Flags Metric Ref     Use Iface 
0.0.0.0         10.23.2.1   0.0.0.0       UG    0      0         0 eth0 
10.23.2.0       0.0.0.0     255.255.255.0 U     1      0         0 eth0
```
#### Desitnation and Genmask Columns
The __Destination column tells you a network prefix, and the Genmask column is the netmask corresponding to that network.__ There are two networks defined in this output: 0.0.0.0/0 (which matches every address on the Internet) and 10.23.2.0/24. 

#### Flags Column
Each network has a U under its Flags column, indicating that the route is active (“up”). 
For 0.0.0.0/0, there is a G in the Flags column, meaning that communication for this network must be sent through the gateway in the __Gateway column__ (10.23.2.1, in this case). However, for 10.23.2.0/24, there is no G in Flags, indicating that the network is directly connected in some way. Here, 0.0.0.0 is used as a stand-in under __Gateway__.

There’s one tricky detail: Say the host wants to send something to 10.23.2.132, which matches both rules in the routing table, 0.0.0.0/0 and 10.23.2.0/24. __How does the kernel know to use the second one?__ It __chooses the longest destination prefix that matches.__ This is where CIDR network form comes in particularly handy: 10.23.2.0/24 matches, and its prefix is 24 bits long; 0.0.0.0/0 also matches, but its prefix is 0 bits long (that is, it has no prefix), so the rule for 10.23.2.0/24 takes priority. 

### The Default Gateway
An entry for 0.0.0.0/0 in the routing table has special significance because it matches any address on the Internet. This is the default route, and the address configured under the Gateway column (in the route -n output) in the default route is the default gateway. __When no other rules match, the default route always does, and the default gateway is where you send messages when there is no other choice__. You can configure a host without a default gateway, but it won’t be able to reach hosts outside the destinations in the routing table. 

## Basic ICMP and DNS Tools
* __Internet Control Message Protocol (ICMP)__ - can help you root out problems with connectivity and routing.
* __Domain Name Service (DNS) system__ - maps names to IP addresses so that you don’t have to remember a bunch of numbers.

### ping
Ping command sends ICMP echo request packets to a host that ask a recipient host to return the packet to the sender. If the recipient host gets the packet and is configured to reply, it sends an ICMP echo response packet in return. For example, say that you run `ping` 10.23.2.1 and get this output: 

`$ ping 10.23.2.1`
```bash
PING 10.23.2.1 (10.23.2.1) 56(84) bytes of data.
64 bytes from 10.23.2.1: icmp_req=1 ttl=64 time=1.76 ms 
64 bytes from 10.23.2.1: icmp_req=2 ttl=64 time=2.35 ms
64 bytes from 10.23.2.1: icmp_req=4 ttl=64 time=1.69 ms
64 bytes from 10.23.2.1: icmp_req=5 ttl=64 time=1.61 ms 
```

The first line says that you’re sending 56-byte packets (84 bytes, if you include the headers) to 10.23.2.1 (by default, one packet per second), and the remaining lines indicate responses from 10.23.2.1. The most important parts of the output are the __sequence number (icmp_req)__ and the __round-trip time (time)__. The number of bytes returned is the size of the packet sent plus 8.

__A gap in the__ *sequence numbers*, __such as the one between 2 and 4, usually means there’s some kind of connectivity problem__. It’s possible for packets to arrive out of order, and if they do, there’s some kind of problem because ping sends only one packet a second. __If a response takes more than a second (1000ms) to arrive, the connection is extremely slow__.

The round-trip time is the total elapsed time between the moment that the request packet leaves and moment that the response packet arrives. __If there’s no way to reach the destination, the final router to see the packet returns an ICMP “host unreachable” packet to ping.__

\*_For security reasons, not all hosts on the Internet respond to ICMP echo request packets, so you might find that you can connect to a website on a host but not get a ping response._

### traceroute (ICMP-based program)
__Use traceroute host to see the path your packets take to a remote host__. (`traceroute -n host` will disable hostname lookups.) One of the best things about traceroute is that it reports return trip times at each step in the route, as demonstrated in this output fragment:

```
5  4.24.203.65  1.312 ms  1.12 ms  1.463 ms  
6  64.159.1.225  1.421 ms  1.37 ms  1.347 ms  
7  64.159.1.38  55.642 ms  55.625 ms  55.663 ms  
8  209.247.10.230  55.89 ms  55.617 ms  55.964 ms  
9  209.244.14.226  55.851 ms 55.726 ms   55.832 ms 
10  209.246.29.174  56.419 ms 56.44 ms   56.423 ms
```
Because this output shows a big latency jump between hops 6 and 7, that part of the route is probably some sort of long-distance link. 

The output from traceroute can be inconsistent. For example, the replies may time out at a certain step, only to “reappear” in later steps. The reason is usually that the router at that step refused to return the debugging output that traceroute wants but routers in later steps were happy to return the output. In addition, a router might choose to assign a lower priority to the debugging traffic than it does to normal traffic.

### DNS and host
IP addresses are difficult to remember and subject to change, which is why we normally use names such as www.example.com instead. The DNS library on your system normally handles this translation automatically, but sometimes you’ll want to __manually translate between a name and an IP address.__ To find the IP address behind a domain name, use the `host` command:

```
$ host www.example.com

www.example.com has address 93.184.216.119 
www.example.com has IPv6 address 2606:2800:220:6d:26bf:1447:1097:aa7
```

You can also __use `host` in reverse__: Enter an IP address instead of a hostname to try to discover the hostname behind the IP address. But __don’t expect this to work reliably__. Many hostnames can represent a single IP address, and DNS doesn’t know how to determine which hostname should correspond to an IP address. The domain administrator must manually set up this reverse lookup, and often the administrator does not.

## The Physical Layer and Ethernet 

The most common kind of physical layer is an Ethernet network. The IEEE 802 family of standards documents defines many different kinds of Ethernet networks, from wired to wireless, but they all have a few things in common, in particular, the following: 
* __All devices on an Ethernet network have a Media Access Control (MAC) address__, sometimes called a hardware address. This address is independent of a host’s IP address, and it is unique to the host’s Ethernet network (but not necessarily a larger software network such as the Internet). A sample MAC address is 10:78:d2:eb:76:97. 
* __Devices on an Ethernet network send messages in frames__, which are wrappers around the data sent. A frame contains the origin and destination MAC addresses. 

Ethernet doesn’t really attempt to go beyond hardware on a single network. For example, if you have two different Ethernet networks with one host attached to both networks (and two different network interface devices), __you can’t directly transmit a frame from one Ethernet network to the other unless you set up a special Ethernet bridge__. And this is where higher network layers (such as the Internet layer) come in. By convention, each Ethernet network is also usually an Internet subnet. __Even though a frame can’t leave one physical network, a router can take the data out of a frame, repackage it, and send it to a host on a different physical network__, which is exactly what happens on the Internet. 

## Understanding Kernel Network Interfaces 
The physical and the Internet layers must be connected in a way that allows the Internet layer to retain its hardware-independent flexibility. __The Linux kernel maintains its own division between the two layers (physical, Internet) and provides communication standards for linking them called a (kernel) network interface. When you configure a network interface, you link the IP address settings from the Internet side with the hardware identification on the physical device side__. Network interfaces have names that usually indicate the kind of hardware underneath, such as eth0 (the first Ethernet card in the computer) and wlan0 (a wireless interface).

In *'Viewing Your Computer’s IP Addresses'*, you learned the most important command for viewing or manually configuring the network interface settings: ifconfig. Recall this output: 
```
eth0      Link encap:Ethernet  HWaddr 10:78:d2:eb:76:97
inet addr:10.23.2.4  Bcast:10.23.2.255  Mask:255.255.255.0           
inet6 addr: fe80::1278:d2ff:feeb:7697/64 Scope:Link           
UP BROADCAST RUNNING MULTICAST MTU:1500   Metric:1           
RX packets:85076006 errors:0 dropped:0 overruns:0 frame:0           
TX packets:68347795 errors:0 dropped:0 overruns:0 carrier:0           
collisions:0 txqueuelen:1000           
RX bytes:86427623613 (86.4 GB)  TX bytes:23437688605 (23.4 GB)           
Interrupt:20 Memory:fe500000-fe520000
```

For each network interface, the left side of the output shows the interface name, and the right side contains settings and statistics for the interface. In addition to the Internet layer pieces that we’ve already covered, you also see __the MAC address on the physical layer (HWaddr)__. The lines containing __UP and RUNNING tell you that the interface is working__. Although ifconfig shows some hardware information (in this case, even some low-level device settings such as the interrupt and memory used), it’s designed primarily for viewing and configuring the software layers attached to the interfaces. To dig deeper into the hardware and physical layer behind a network interface, __use something like the `ethtool` command to display or change the settings on Ethernet cards__. (We’ll look briefly at wireless networks in 9.23 Wireless Ethernet.)

## Introduction to Network Interface Configuration
You’ve now seen all of the basic elements that go into the lower levels of a network stack: the physical layer, the network (Internet) layer, and the Linux kernel’s network interfaces. __In order to__ combine these pieces to __connect a Linux machine to the Internet, you or a piece of software must do the following:__

1. Connect the network hardware and __ensure that the kernel has a driver for it. If the driver is present, `ifconfig -a` displays a kernel network interface corresponding to the hardware__. 
2. Perform any additional physical layer setup, such as choosing a network name or password. 
3. __Bind an IP address and netmask to the kernel network interface__ so that the kernel’s device drivers (physical layer) and Internet subsystems (Internet layer) can talk to each other. 
4. Add any additional necessary routes, including the default gateway. 

When all machines were big stationary boxes wired together, this was relatively straightforward: The kernel did step 1, you didn’t need step 2, and you’d do step 3 with the ifconfig command and step 4 with the route command. 

To manually set the IP address and netmask for a kernel network interface, you’d do this: 

`$ sudo ifconfig <interface> <address> <netmask> <mask>`

Here, interface is the name of the `interface`, such as eth0. When the interface was up, you’d be ready to add routes, which was typically just a matter of setting the `default gateway`, like this:

`$ sudo route add default gw <gw-address>`

__The gw-address parameter is the IP address of your default gateway; it must be an address in a locally connected subnet defined by the address and mask settings of one of your network interfaces.__

### Manually Adding and Deleting Routes
To remove a default gateway, run 

` sudo route del -net default`

You can easily override the default gateway with other routes. For example, say your machine is on subnet 10.23.2.0/24, you want to reach a subnet at 192.168.45.0/24, and you know that 10.23.2.44 can act as a router for that subnet. Run this command to send traffic bound for 192.168.45.0 to that router:

`$ sudo route add -net 192.168.45.0/24 gw 10.23.2.44`

You don’t need to specify the router in order to delete a route: 

`$ sudo route del -net 192.168.45.0/24`

Now, before you go crazy with routes, you should know that messing with routes is often more complicated than it appears. For this particular example, you also have to make sure that the routing for all hosts on 192.163.45.0/24 can lead back to 10.23.2.0/24, or the first route you add is basically useless.

Normally, you should keep things as simple as possible for your clients, setting up networks so that their hosts need only a default route. If you need multiple subnets and the ability to route between them, it’s usually best to configure the routers acting as the default gateways to do all of the work of routing between different local subnets. (You’ll see an example in 9.17 Configuring Linux as a Router.) 

## Boot-Activated Network Configuration 
We’ve discussed ways to manually configure a network, and the traditional way to ensure the correctness of a machine’s network configuration was to __have init run a script to run the manual configuration at boot time__. This boils down to running tools like ifconfig and route somewhere in the chain of boot events. Many servers still do it this way.

There have been many attempts in Linux to standardize configuration files for boot-time networking. The tools `ifup` and `ifdown` do so—for example, a boot script can (in theory) run ifup eth0 to run the correct `ifconfig` and `route` commands for the eth0 interface. Unfortunately, different distributions have completely different implementations of `ifup` and `ifdown`, and as a result, their configuration files are also completely different. Ubuntu, for example, uses the ifupdown suite with configuration files in */etc/network*, and Fedora uses its own set of scripts with configuration in */etc/sysconfig/network-scripts*. 

You don’t need to know the details of these configuration files, and if you insist on doing it all by hand and bypass your distribution’s configuration tools, you can just look up the formats in manual pages such as ifup(8) and interfaces(5). __But it is important to know that this type of boot-activated configuration is often not even used. You’ll most often see it for the local-host (or lo; see 9.13 Localhost) network interface but nothing else because it’s too inflexible to meet the needs of modern systems.__

## Problems with Manual and Boot-Activated Network Configuration 
Although most systems used to configure the network in their boot mechanisms—and many servers still do—the dynamic nature of modern networks means that most machines don’t have static (unchanging) IP addresses. Rather than storing the IP address and other network information on your machine, your machine gets this information from somewhere on the local physical network when it first attaches to that network. Most normal network client applications don’t particularly care what IP address your machine uses, as long as it works. Dynamic Host Configuration Protocol (DHCP, described in 9.16 Understanding DHCP) tools do the basic network layer configuration on typical clients. 

There’s more to the story, though. For example, wireless networks add additional dimensions to interface configuration, such as network names, authentication, and encryption techniques. When you step back to look at the bigger picture, you see that your system needs a way to answer the following questions:

* If the machine has multiple physical network interfaces (such as a notebook with wired and wireless Ethernet), how do you choose which one(s) to use?
*How should the machine set up the physical interface? For wireless networks, this includes scanning for network names, choosing a name, and negotiating authentication.
* Once the physical network interface is connected, how should the machine set up the software network layers, such as the Internet layer?
* How can you let a user choose connectivity options? For example, how do you let a user choose a wireless network?
* What should the machine do if it loses connectivity on a network interface?

Answering these questions is usually more than simple boot scripts can handle, and it’s a real hassle to do it all by hand. The answer is to use a system service that can monitor physical networks and choose (and automatically configure) the kernel network interfaces based on a set of rules that makes sense to the user. The service should also be able to respond to requests from users, who should be able to change the wireless network they’re on without having to become root just to fiddle around with network settings every time something changes. 

## Network Configuration Managers
There are several ways to automatically configure networks in Linux-based systems. The most widely used option on desktops and notebooks is NetworkManager. Other network configuration management systems are mainly targeted for smaller embedded systems, such as OpenWRT’s `netifd`, Android’s ConnectivityManager service, ConnMan, and Wicd. We’ll briefly discuss NetworkManager because it’s the one you’re most likely to encounter. We won’t go into a tremendous amount of detail, though, because after you see the big picture, NetworkManager and other configuration systems will be more transparent. 

### NetworkManager Operation
NetworkManager is a daemon that the system starts upon boot. Like all daemons, it does not depend on a running desktop component. __Its job is to listen to events from the system and users and to change the network configuration based on a bunch of rules.__ 

When running, __NetworkManager maintains two basic levels of configuration. The first is a collection of information about available hardware devices__, which it normally collects from the kernel and maintains by monitoring udev over the Desktop Bus (D-Bus). __The second configuration level is a more specific list of connections: hardware devices and additional physical and network layer configuration parameters.__ For example, a wireless network can be represented as a connection. 

__To activate a connection, NetworkManager often delegates the tasks to other specialized network tools and daemons such as dhclient to get Internet layer configuration from a locally attached physical network__. Because network configuration tools and schemes vary among distributions, NetworkManager uses plugins to interface with them, rather than imposing its own standard. There are plugins for the both the Debian/ Ubuntu and Red Hat–style interface configuration, for example. 

__Upon startup, NetworkManager gathers all available network device information, searches its list of connections, and then decides to try to activate one__. Here’s how it makes that decision for Ethernet interfaces: 

1. If a wired connection is available, try to connect using it. Otherwise, try the wireless connections. 
2. Scan the list of available wireless networks. If a network is available that you’ve previously connected to, NetworkManager will try it again. 
3. If more than one previously connected wireless networks are available, select the most recently connected. 

After establishing a connection, NetworkManager maintains it until the connection is lost, a better network becomes available (for example, you plug in a network cable while connected over wireless), or the user forces a change. 

### Interacting with NetworkManager
Most users interact with NetworkManager through an applet on the desktop—it’s usually an icon in the upper or lower right that indicates the connection status (wired, wireless, or not connected). When you click on the icon, you get a number of connectivity options, such as a choice of wireless networks and an option to disconnect from your current network. Each desktop environment has its own version of this applet, so it looks a little different on each one.

In addition to the applet, __there are a few tools that you can use to query and control NetworkManager from your shell. For a very quick summary of your current connection status, use the `nm-tool` command with no arguments. You’ll get a list of interfaces and configuration parameters.__ In some ways, this is like ifconfig except that there’s more detail, especially when viewing wireless connections.

__To control NetworkManager from the command line, use the `nmcli` command.__ This is a somewhat extensive command. See the nmcli(1) manual page for more information. 

__Finally, the utility `nm-online` will tell you whether the network is up or down__. If the network is up, the command returns zero as its exit code; it’s nonzero otherwise. (For more on how to use an exit code in a shell script, see Chapter 11.) 

### NetworkManager Configuration
The general configuration directory for NetworkManager is usually /etc/NetworkManager, and there are several different kinds of configuration. The general configuration file is NetworkManager.conf. The format is similar to the XDG-style .desktop and Microsoft .ini files, with key-value parameters falling into different sections. __You’ll find that nearly every configuration file has a [main] section that defines the plugins to use.__ Here’s a simple example that activates the ifupdown plugin used by Ubuntu and Debian: 

```
[main] 
plugins=ifupdown,keyfile
```

Other distribution-specific plugins are ifcfg-rh (for Red Hat–style distributions) and ifcfg-suse (for SuSE). The keyfile plugin that you also see here supports NetworkManager’s native configuration file support. When using the plugin, you can see the system’s known connections in /etc/NetworkManager/system-connections. 

For the most part, you won’t need to change NetworkManager.conf because the more specific configuration options are found in other files. 

#### Unmanaged Interfaces 
Although you may want NetworkManager to manage most of your network interfaces, there will be times when you want it to ignore interfaces. For example, there’s no reason why most users would need any kind of dynamic configuration on the localhost (lo) interface because the configuration never changes. You also want to configure this interface early in the boot process because basic system services often depend on it. Most distributions keep NetworkManager away from localhost. 

You can tell NetworkManager to disregard an interface by using plugins. If you’re using the ifupdown plugin (for example, in Ubuntu and Debian), add the interface configuration to your /etc/network/interfaces file and then set the value of managed to false in the ifupdown section of the NetworkManager.conf file: 

```
[ifupdown]
managed=false
```

For the ifcfg-rh plugin that Fedora and Red Hat use, look for a line like this in the /etc/sysconfig/network-scripts directory that contains the ifcfg-* configuration files:

```
NM_CONTROLLED=yes
``` 

If this line is not present or the value is set to no, NetworkManager ignores the interface. For example, you’ll find it deactivated in the ifcfg-lo file. You can also specify a hardware address to ignore, like this:

```
HWADDR=10:78:d2:eb:76:97
```

If you don’t use either of these network configuration schemes, you can still use the keyfile plugin to specify the unmanaged device directly inside your NetworkManager.conf file using the MAC address. Here’s how that might look:

```[keyfile]
unmanaged-devices=mac:10:78:d2:eb:76:97;mac:1c:65:9d:cc:ff:b9
```